# The effect of sequencing depth
This notebook contains the analysis of the effect of sequencing depth on pan-genome results.  
Pan-genomes are compared to a "high-quality" (HQ) pan-genome, constructed by running Panoramic with HQ assemblies and annotation-evidence. Pan-genomes were constructed from the full data of [PRJEB31147](https://www.ebi.ac.uk/ena/browser/view/PRJEB31147) or from sub-samples of it (x10, x50 etc).

In [ ]:
import os
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from Bio import SeqIO
import numpy as np
from scipy.stats import ttest_ind
from itertools import chain

## Paths

In [ ]:
# de novo pan-genomes
dn_pan_genomes = {
    'HQ-assembly': "../output/A_thaliana_pan_genome/de_novo/HQ_assembly/RESULT",
    "full-data": "../output/A_thaliana_pan_genome/de_novo/full_data/RESULT",
    "x50": "../output/A_thaliana_pan_genome/de_novo/x50/RESULT",
    "x30": "../output/A_thaliana_pan_genome/de_novo/x30/RESULT",
    "x20": "../output/A_thaliana_pan_genome/de_novo/x20/RESULT",
    "x10": "../output/A_thaliana_pan_genome/de_novo/x10/RESULT",
}

In [ ]:
# map-to-pan pan-genomes
mtp_pan_genomes = {
    'HQ-assembly': "../output/A_thaliana_pan_genome/map_to_pan/HQ_assembly/RESULT",
    "full-data": "../output/A_thaliana_pan_genome/map_to_pan/full_data/RESULT",
    "x50": "../output/A_thaliana_pan_genome/map_to_pan/x50/RESULT",
    "x30": "../output/A_thaliana_pan_genome/map_to_pan/x30/RESULT",
    "x20": "../output/A_thaliana_pan_genome/map_to_pan/x20/RESULT",
    "x10": "../output/A_thaliana_pan_genome/map_to_pan/x10/RESULT",
}

In [ ]:
# de novo comparison dirs
dn_compare = {
    "full-data": "../output/A_thaliana_pan_genome/compare_pan_genomes/DN_HQ_asm_vs_DN_full_data/RESULT",
    "x50": "../output/A_thaliana_pan_genome/compare_pan_genomes/DN_HQ_asm_vs_DN_x50/RESULT",
    "x30": "../output/A_thaliana_pan_genome/compare_pan_genomes/DN_HQ_asm_vs_DN_x30/RESULT",
    "x20": "../output/A_thaliana_pan_genome/compare_pan_genomes/DN_HQ_asm_vs_DN_x20/RESULT",
    "x10": "../output/A_thaliana_pan_genome/compare_pan_genomes/DN_HQ_asm_vs_DN_x10/RESULT",
}

In [ ]:
# map-to-pan comparison dirs
mtp_compare = {
    "full-data": "../output/A_thaliana_pan_genome/compare_pan_genomes/MTP_HQ_asm_vs_MTP_full_data/RESULT",
    "x50": "../output/A_thaliana_pan_genome/compare_pan_genomes/MTP_HQ_asm_vs_MTP_x50/RESULT",
    "x30": "../output/A_thaliana_pan_genome/compare_pan_genomes/MTP_HQ_asm_vs_MTP_x30/RESULT",
    "x20": "../output/A_thaliana_pan_genome/compare_pan_genomes/MTP_HQ_asm_vs_MTP_x20/RESULT",
    "x10": "../output/A_thaliana_pan_genome/compare_pan_genomes/MTP_HQ_asm_vs_MTP_x10/RESULT",
}

## Assembly stats
Assemblies are common to the DN and MTP pan-genomes, so no need to present results for both

In [ ]:
pg_order = ["x10", "x20", "x30", "x50","full-data", "HQ-assembly"]

In [ ]:
dn_pg_asm_stats = {pg: pd.read_csv(os.path.join(dn_pan_genomes[pg],"all_samples/stats/assembly_stats.tsv"),
                                    sep='\t', index_col=0) for pg in pg_order[:-1]}

In [ ]:
keep_columns = ['Input bases', 'Clean bases', '# contigs (>= 0 bp)', 'Total length (>= 0 bp)',
               'N50', '% Complete BUSCOs', '% unmapped (Chr0)']
for pg in dn_pg_asm_stats:
    dn_pg_asm_stats[pg] = dn_pg_asm_stats[pg][keep_columns]

In [ ]:
n50_dfs = []
for pg in pg_order[:-1]:
    tmp = pd.DataFrame(dn_pg_asm_stats[pg]['N50'])
    tmp = tmp.reset_index()
    tmp.columns = ['sample','N50']
    tmp['PG'] = pg
    n50_dfs.append(tmp)
n50_df = pd.concat(n50_dfs)

In [ ]:
fig = px.strip(n50_df, x='PG', y='N50', color='sample', title='Contig N50')
fig.update_xaxes(title_text = '')
fig.show()

In [ ]:
assembly_size_dfs = []
for pg in pg_order[:-1]:
    tmp = pd.DataFrame(dn_pg_asm_stats[pg]['Total length (>= 0 bp)'])
    tmp = tmp.reset_index()
    tmp.columns = ['sample','Assembly size']
    tmp['PG'] = pg
    assembly_size_dfs.append(tmp)
assembly_size_df = pd.concat(assembly_size_dfs)

In [ ]:
fig = px.strip(assembly_size_df, x='PG', y='Assembly size', color='sample', title='Assembly size')
fig.update_xaxes(title_text = '')
fig.show()

## Pan genome size and composition

### Occupancy categories

In [ ]:
# read pan_PAV.tsv files as data frames
dn_pg_pav = {pg: pd.read_csv(os.path.join(dn_pan_genomes[pg],"all_samples/pan_genome/pan_PAV.tsv"), sep='\t', index_col='gene')
             for pg in pg_order}
mtp_pg_pav = {pg: pd.read_csv(os.path.join(mtp_pan_genomes[pg],"all_samples/pan_genome/pan_PAV.tsv"), sep='\t', index_col='gene')
             for pg in pg_order}

In [ ]:
def count_catgories(pav_df):
    """
    Given a PAV df, return [core genes, shell genes, singletons]
    """
    n_samples = pav_df.shape[1]
    occup_counts = pav_df.sum(axis=1).value_counts()
    core = occup_counts.loc[n_samples]
    singletons = occup_counts.loc[1]
    shell = occup_counts.loc[(occup_counts.index > 1) & (occup_counts.index < n_samples)].sum()
    return [core, shell, singletons]

In [ ]:
dn_pg_sizes = {pg: count_catgories(dn_pg_pav[pg]) for pg in pg_order}
mtp_pg_sizes = {pg: count_catgories(mtp_pg_pav[pg]) for pg in pg_order}

In [ ]:
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=("De novo", "Map-to-pan"),
                   vertical_spacing=0.1)
fig.add_trace(go.Bar(name='Core', x=pg_order, y=[dn_pg_sizes[pg][0] for pg in pg_order], marker_color='darkgreen', legendgroup='core'), row=1, col=1)
fig.add_trace(go.Bar(name='Shell', x=pg_order, y=[dn_pg_sizes[pg][1] for pg in pg_order], marker_color='orange', legendgroup='shell'), row=1, col=1)
fig.add_trace(go.Bar(name='Singletons', x=pg_order, y=[dn_pg_sizes[pg][2] for pg in pg_order], marker_color='red', legendgroup='singletons'), row=1, col=1)
fig.add_trace(go.Bar(name='Core', x=pg_order, y=[mtp_pg_sizes[pg][0] for pg in pg_order], marker_color='darkgreen', legendgroup='core', showlegend=False), row=2, col=1)
fig.add_trace(go.Bar(name='Shell', x=pg_order, y=[mtp_pg_sizes[pg][1] for pg in pg_order], marker_color='orange', legendgroup='shell', showlegend=False), row=2, col=1)
fig.add_trace(go.Bar(name='Singletons', x=pg_order, y=[mtp_pg_sizes[pg][2] for pg in pg_order], marker_color='red', legendgroup='singletons', showlegend=False), row=2, col=1)
fig.update_layout(barmode='stack')
fig.update_yaxes(title_text = "# of genes")
fig.update_layout(height=700, width=1000, title_text="Pan-genome composition")
fig.show()

In [ ]:
dn_pg_sizes_df = pd.DataFrame(dn_pg_sizes)
dn_pg_sizes_df.index = ['Core','Shell','Singletons']
dn_pg_sizes_df = dn_pg_sizes_df.transpose()
dn_pg_sizes_df['Total'] = dn_pg_sizes_df.sum(axis=1)

mtp_pg_sizes_df = pd.DataFrame(mtp_pg_sizes)
mtp_pg_sizes_df.index = ['Core','Shell','Singletons']
mtp_pg_sizes_df = mtp_pg_sizes_df.transpose()
mtp_pg_sizes_df['Total'] = mtp_pg_sizes_df.sum(axis=1)

pg_sizes = dn_pg_sizes_df.join(mtp_pg_sizes_df, rsuffix='_MTP')
pg_sizes.columns = pd.MultiIndex.from_product([['De novo','Map-to-pan'],['Core','Shell','Singletons','Total']])
pg_sizes

### Occupancy distributions

In [ ]:
def occup_stats(pav_df):
    occups = list(pav_df.sum(axis=1))
    return (np.mean(occups), np.std(occups))

In [ ]:
dn_pg_occup_stats = {pg: occup_stats(dn_pg_pav[pg]) for pg in pg_order}
mtp_pg_occup_stats = {pg: occup_stats(mtp_pg_pav[pg]) for pg in pg_order}

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=pg_order, y=[dn_pg_occup_stats[pg][0] for pg in pg_order],
    error_y=dict(type='data', array=[dn_pg_occup_stats[pg][1] for pg in pg_order]),
    name='De novo', marker_color='lightblue'
))
fig.add_trace(go.Bar(
    x=pg_order, y=[mtp_pg_occup_stats[pg][0] for pg in pg_order],
    error_y=dict(type='data', array=[mtp_pg_occup_stats[pg][1] for pg in pg_order]),
    name='Map-to-pan', marker_color='darkslateblue'
))
fig.update_yaxes(title_text = "Occupancy")
fig.update_layout(title_text="Mean occupancy")

fig.show()

In [ ]:
dn_occup_df = pd.DataFrame(dn_pg_occup_stats).transpose()
dn_occup_df.columns = ['Mean','STD']
mtp_occup_df = pd.DataFrame(mtp_pg_occup_stats).transpose()
mtp_occup_df.columns = ['Mean','STD']

occup_df = dn_occup_df.join(mtp_occup_df, rsuffix='MTP_')
occup_df.columns = pd.MultiIndex.from_product([['De novo','Map-to-pan'],['Mean','STD']])
occup_df

## Genes per sample
Number of genes detected as present in each sample, across data sets and pipelines

In [ ]:
# generate df
dn_gene_counts = pd.concat([dn_pg_pav[pg].sum() for pg in pg_order], axis=1)
dn_gene_counts.columns = pg_order
mtp_gene_counts = pd.concat([mtp_pg_pav[pg].sum() for pg in pg_order], axis=1)
mtp_gene_counts.columns = pg_order

gene_counts = dn_gene_counts.join(mtp_gene_counts, rsuffix='MTP_')
gene_counts.columns = pd.MultiIndex.from_product([['De novo','Map-to-pan'],pg_order])
gene_counts

In [ ]:
dn_gene_counts_melt = dn_gene_counts.reset_index().melt(id_vars='index', value_vars=pg_order)
dn_gene_counts_melt.columns = ['sample','PG','count']
mtp_gene_counts_melt = mtp_gene_counts.reset_index().melt(id_vars='index', value_vars=pg_order)
mtp_gene_counts_melt.columns = ['sample','PG','count']

In [ ]:
fig = px.strip(dn_gene_counts_melt, x='PG', y='count', color='sample',
               title='De-novo - number of genes per sample',
              range_y=[18000,28000])
fig.update_yaxes(title_text = "# of genes")
fig.show()

In [ ]:
fig = px.strip(mtp_gene_counts_melt, x='PG', y='count', color='sample',
               title='Map-to-pan - number of genes per sample',
              range_y=[18000,28000])
fig.update_yaxes(title_text = "# of genes")
fig.show()

## Nonreference gene pool
Exploring the nonreference gene pool of each pan-genome by comparing it to the HQ-assembly pan-genome.

### Matched/Unmatched proportions

In [ ]:
pg_order = ["x10", "x20", "x30", "x50","full-data"]

In [ ]:
# How many non-ref genes per PG and how many of these match HQ non-ref
dn_nonref_matches = {pg: pd.read_csv(os.path.join(dn_compare[pg],"A_thaliana_DN_%s_vs_A_thaliana_DN_HQ_asm_max_weight_matches.tsv" % pg.replace('-','_')), sep='\t')
             for pg in pg_order}
mtp_nonref_matches = {pg: pd.read_csv(os.path.join(mtp_compare[pg],"A_thaliana_MTP_%s_vs_A_thaliana_MTP_HQ_asm_max_weight_matches.tsv" % pg.replace('-','_')), sep='\t')
             for pg in pg_order}

In [ ]:
def count_nonref(pg1_pav_df, pg2_pav_df, matches_df):
    pg1_nonref = pg1_pav_df.index.str.startswith('PanGene').sum()
    pg2_nonref = pg2_pav_df.index.str.startswith('PanGene').sum()
    matched_nonref = matches_df.shape[0]
    return (matched_nonref, pg1_nonref - matched_nonref, pg2_nonref - matched_nonref)

In [ ]:
dn_nonref_counts = {pg: count_nonref(dn_pg_pav[pg], dn_pg_pav['HQ-assembly'], dn_nonref_matches[pg]) for pg in pg_order}
mtp_nonref_counts = {pg: count_nonref(mtp_pg_pav[pg], mtp_pg_pav['HQ-assembly'], mtp_nonref_matches[pg]) for pg in pg_order}

In [ ]:
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=("De novo", "Map-to-pan"),
                   vertical_spacing=0.1)

fig.add_trace(go.Bar(name='Matched', x=pg_order, y=[dn_nonref_counts[pg][0] for pg in pg_order], legendgroup='matched', marker_color='royalblue'), row=1, col=1)    ,
fig.add_trace(go.Bar(name='PG+/HQ-', x=pg_order, y=[dn_nonref_counts[pg][1] for pg in pg_order], legendgroup='PG+/HQ-', marker_color='tomato'), row=1, col=1)
fig.add_trace(go.Bar(name='PG-/HQ+', x=pg_order, y=[dn_nonref_counts[pg][2] for pg in pg_order], legendgroup='PG-/HQ+', marker_color='sandybrown'), row=1, col=1)
fig.add_trace(go.Bar(name='Matched', x=pg_order, y=[mtp_nonref_counts[pg][0] for pg in pg_order], legendgroup='matched', showlegend=False, marker_color='royalblue'), row=2, col=1)    ,
fig.add_trace(go.Bar(name='PG+/HQ-', x=pg_order, y=[mtp_nonref_counts[pg][1] for pg in pg_order], legendgroup='PG+/HQ-', showlegend=False, marker_color='tomato'), row=2, col=1)
fig.add_trace(go.Bar(name='PG-/HQ+', x=pg_order, y=[mtp_nonref_counts[pg][2] for pg in pg_order], legendgroup='PG-/HQ+', showlegend=False, marker_color='sandybrown'), row=2, col=1)


# Change the bar mode
fig.update_layout(barmode='stack')
fig.update_yaxes(title_text = "# of genes")
fig.update_layout(title_text="Number of nonreference genes")
fig.show()

In [ ]:
dn_nonref_df = pd.DataFrame(dn_nonref_counts).transpose()
dn_nonref_df.columns = ['Matched', 'PG+/HQ-', 'PG-/HQ+']
mtp_nonref_df = pd.DataFrame(mtp_nonref_counts).transpose()
mtp_nonref_df.columns = ['Matched', 'PG+/HQ-', 'PG-/HQ+']
nonref_df = dn_nonref_df.join(mtp_nonref_df, rsuffix='MTP_')
nonref_df.columns = pd.MultiIndex.from_product([['De novo','Map-to-pan'],['Matched', 'PG+/HQ-', 'PG-/HQ+']])
nonref_df

### Matched/Unmatched occupancies
Comapre occupancies of matched and unmatched nonreference genes

In [ ]:
def nonref_occup(pg1_pav, pg2_pav, matches_df):
    pg1_matched = set(matches_df.iloc[:,0])
    pg2_matched = set(matches_df.iloc[:,1])
    ref_occup_pg1 = pg1_pav.loc[(~ pg1_pav.index.str.startswith('PanGene'))].sum(axis=1) - 1
    ref_occup_pg2 = pg2_pav.loc[(~ pg2_pav.index.str.startswith('PanGene'))].sum(axis=1) - 1
    matched_occup_pg1 = pg1_pav.loc[(pg1_pav.index.str.startswith('PanGene')) & (pg1_pav.index.isin(pg1_matched))].sum(axis=1)
    matched_occup_pg2 = pg2_pav.loc[(pg2_pav.index.str.startswith('PanGene')) & (pg2_pav.index.isin(pg2_matched))].sum(axis=1)
    unmatched_occup_pg1 = pg1_pav.loc[(pg1_pav.index.str.startswith('PanGene')) & (~ pg1_pav.index.isin(pg1_matched))].sum(axis=1)
    unmatched_occup_pg2 = pg2_pav.loc[(pg2_pav.index.str.startswith('PanGene')) & (~ pg2_pav.index.isin(pg2_matched))].sum(axis=1)
    return [ref_occup_pg1, ref_occup_pg2, matched_occup_pg1, matched_occup_pg2, unmatched_occup_pg1, unmatched_occup_pg2]

In [ ]:
dn_occup_nonref = {pg : nonref_occup(dn_pg_pav[pg], dn_pg_pav['HQ-assembly'], dn_nonref_matches[pg]) for pg in pg_order}
mtp_occup_nonref = {pg : nonref_occup(mtp_pg_pav[pg], mtp_pg_pav['HQ-assembly'], mtp_nonref_matches[pg]) for pg in pg_order}

In [ ]:
fig_dict = {
    0 : ['Reference genes - PG', 'lightblue'],
    1 : ['Reference genes - HQ', 'lightgreen'],
    2 : ['Nonreference genes - matched - PG', 'tomato'],
    3 : ['Nonreference genes - matched - HQ', 'orange'],
    4 : ['Nonreference genes - unmatched - PG', 'grey'],
    5 : ['Nonreference genes - unmatched - HQ', 'mediumorchid'],
}

fig = go.Figure()
for i in range(6):
    fig.add_trace(go.Bar(
        x=pg_order, y=[dn_occup_nonref[pg][i].mean() for pg in pg_order],
        error_y=dict(type='data', array=[dn_occup_nonref[pg][i].std() for pg in pg_order]),
        name=fig_dict[i][0], marker_color=fig_dict[i][1]
))

fig.update_yaxes(title_text = "Occupancy")
fig.update_layout(title_text="Mean occupancy - De novo")

fig.show()

In [ ]:
fig = go.Figure()
for i in range(6):
    fig.add_trace(go.Bar(
        x=pg_order, y=[mtp_occup_nonref[pg][i].mean() for pg in pg_order],
        error_y=dict(type='data', array=[dn_occup_nonref[pg][i].std() for pg in pg_order]),
        name=fig_dict[i][0], marker_color=fig_dict[i][1]
))

fig.update_yaxes(title_text = "Occupancy")
fig.update_layout(title_text="Mean occupancy - Map-to-pan")

fig.show()

### Matched/Unmatched protein lengths

In [ ]:
def prot_lens(pg, nonref_fasta, matches_df):
    matched = set(matches_df.iloc[:,0])
    lens = []
    for rec in SeqIO.parse(nonref_fasta, 'fasta'):
        l = len(rec.seq)
        recid = rec.id
        if recid in matched:
            stat = 'matched'
        else:
            stat = 'unmatched'
        d = {'PG': pg, 'prot': recid, 'length': l, 'status': stat}
        lens.append(d)
    return pd.DataFrame(lens)

In [ ]:
dn_nonref_prot_lens = {pg: prot_lens(pg, dn_compare[pg] + '/A_thaliana_DN_%s_nonref.fasta' % pg.replace('-','_'), dn_nonref_matches[pg]) for pg in pg_order}
mtp_nonref_prot_lens = {pg: prot_lens(pg, mtp_compare[pg] + '/A_thaliana_MTP_%s_nonref.fasta' % pg.replace('-','_'), mtp_nonref_matches[pg]) for pg in pg_order}

In [ ]:
dn_lens_df = pd.concat(dn_nonref_prot_lens)
mtp_lens_df = pd.concat(mtp_nonref_prot_lens)

In [ ]:
fig = px.box(dn_lens_df, x='PG', y="length", color='status', title='De novo - nonreference protein lengths',
            color_discrete_sequence=['tomato','royalblue'])
fig.update_yaxes(title_text = "Protein length (AA)")
fig.update_xaxes(title_text = '')
fig.update_layout(legend_traceorder="reversed")
fig.show()

In [ ]:
fig = px.box(mtp_lens_df, x='PG', y="length", color='status', title='Map-to-pan - nonreference protein lengths',
            color_discrete_sequence=['tomato','royalblue'])
fig.update_yaxes(title_text = "Protein length (AA)")
fig.update_xaxes(title_text = '')
fig.update_layout(legend_traceorder="reversed")
fig.show()

In [ ]:
def prot_len_diff(lens_df, pg):
    matched = lens_df.loc[(lens_df['PG'] == pg) & (lens_df['status'] == "matched")]['length']
    unmatched = lens_df.loc[(lens_df['PG'] == pg) & (lens_df['status'] == "unmatched")]['length']
    matched_mean = np.mean(matched)
    unmatched_mean = np.mean(unmatched)
    matched_median = np.median(matched)
    unmatched_median = np.median(unmatched)
    t,p = ttest_ind(matched, unmatched)
    return [matched_mean,unmatched_mean,matched_median,unmatched_median,t,p]

In [ ]:
cols = ['Matched mean','Unmatched mean','Matched median','Unmatched median','T','p']

dn_prot_len_diff = {pg: prot_len_diff(dn_lens_df, pg) for pg in pg_order}
dn_prot_len_diff_df = pd.DataFrame(dn_prot_len_diff).transpose()
dn_prot_len_diff_df.columns = cols

mtp_prot_len_diff = {pg: prot_len_diff(mtp_lens_df, pg) for pg in pg_order}
mtp_prot_len_diff_df = pd.DataFrame(mtp_prot_len_diff).transpose()
mtp_prot_len_diff_df.columns = cols

prot_len_diff_df = dn_prot_len_diff_df.join(mtp_prot_len_diff_df, rsuffix='MTP_')
prot_len_diff_df.columns = pd.MultiIndex.from_product([['De novo','Map-to-pan'],cols])

prot_len_diff_df

### Unmatched nonreference gene mapping
To better understand the origin of unmatched nonreference genes, transcripts of such genes were mapped to all  assemblies in the other pan-genome, and the number of transcripts that could not be mapped (95% transcript sequence coverage) to any assembly was calculated.  
It is assumed that unmatched transcripts that could not be mapped originate from the absence of the relevant sequences in the assembly, whereas mapped transcripts indicate another source, e.g. gene duplications or clustering issues.

In [ ]:
colnames = ['sample','chrom','start','end']
dn_unmatched_mapping = {pg: [pd.read_csv(os.path.join(dn_compare[pg],"A_thaliana_DN_%s_vs_A_thaliana_DN_HQ_asm_nonref_unmatched_mapped.tsv" % pg.replace('-','_')), sep='\t', names=colnames),
                             pd.read_csv(os.path.join(dn_compare[pg],"A_thaliana_DN_HQ_asm_vs_A_thaliana_DN_%s_nonref_unmatched_mapped.tsv" % pg.replace('-','_')), sep='\t', names=colnames)]
                        for pg in pg_order}
mtp_unmatched_mapping = {pg: [pd.read_csv(os.path.join(mtp_compare[pg],"A_thaliana_MTP_%s_vs_A_thaliana_MTP_HQ_asm_nonref_unmatched_mapped.tsv" % pg.replace('-','_')), sep='\t', names=colnames),
                             pd.read_csv(os.path.join(mtp_compare[pg],"A_thaliana_MTP_HQ_asm_vs_A_thaliana_MTP_%s_nonref_unmatched_mapped.tsv" % pg.replace('-','_')), sep='\t', names=colnames)]
                        for pg in pg_order}

In [ ]:
# return [# of mapped, # of unmapped]
def unmatched_mapping(mapping_df):
    tmp = mapping_df['sample'].isna().value_counts()
    return [tmp[False], tmp[True]]

In [ ]:
dn_unmatched_mapping_counts = {pg: [unmatched_mapping(dn_unmatched_mapping[pg][0]), unmatched_mapping(dn_unmatched_mapping[pg][1])] for pg in pg_order} 
mtp_unmatched_mapping_counts = {pg: [unmatched_mapping(mtp_unmatched_mapping[pg][0]), unmatched_mapping(mtp_unmatched_mapping[pg][1])] for pg in pg_order} 

In [ ]:
x = [
    list(chain(*[[pg]*2 for pg in pg_order])),
    ['PG','HQ']*len(pg_order)
]
y1 = list(chain(*[[dn_unmatched_mapping_counts[pg][0][0], dn_unmatched_mapping_counts[pg][1][0]] for pg in pg_order]))
y2 = list(chain(*[[dn_unmatched_mapping_counts[pg][0][1], dn_unmatched_mapping_counts[pg][1][1]] for pg in pg_order]))
fig = go.Figure()
fig.add_bar(name="Mapped", x=x, y=y1)
fig.add_bar(name="Unmapped", x=x, y=y2)
fig.update_layout(barmode="relative", title_text="De novo - Unmatched nonreference genes")
fig.update_yaxes(title_text="# of genes")
fig.show()

In [ ]:
x = [
    list(chain(*[[pg]*2 for pg in pg_order])),
    ['PG','HQ']*len(pg_order)
]
y1 = list(chain(*[[mtp_unmatched_mapping_counts[pg][0][0], mtp_unmatched_mapping_counts[pg][1][0]] for pg in pg_order]))
y2 = list(chain(*[[mtp_unmatched_mapping_counts[pg][0][1], mtp_unmatched_mapping_counts[pg][1][1]] for pg in pg_order]))
fig = go.Figure()
fig.add_bar(name="Mapped", x=x, y=y1)
fig.add_bar(name="Unmapped", x=x, y=y2)
fig.update_layout(barmode="relative", title_text="Map-to-pan - Unmatched nonreference genes")
fig.update_yaxes(title_text="# of genes")
fig.show()

## Discrepancies
Analyze discrepancies between each PG and the HQ PG

In [ ]:
# read discrepancies tsvs
dn_discrep = {pg: pd.read_csv(os.path.join(dn_compare[pg],"discrepancies.tsv"), sep='\t', index_col='gene')
             for pg in pg_order}
mtp_discrep = {pg: pd.read_csv(os.path.join(mtp_compare[pg],"discrepancies.tsv"), sep='\t', index_col='gene')
             for pg in pg_order}

### Discrpancy type
PG+/HQ- or PG-/HQ+

In [ ]:
# {PG : [-1 count, 1 count] }
def count_discrep_types(df):
    type_1 = len(df.loc[df['type']==1])
    type_minus1 = len(df.loc[df['type']==-1])
    return [type_minus1, type_1]

In [ ]:
dn_discrep_types = {pg: count_discrep_types(dn_discrep[pg]) for pg in pg_order}
mtp_discrep_types = {pg: count_discrep_types(mtp_discrep[pg]) for pg in pg_order}

In [ ]:
# Only consider matched genes (ref + matched non-ref)
# {PG : [-1 count, 1 count] }
dn_discrep_types_matched = {pg: count_discrep_types(dn_discrep[pg].loc[~dn_discrep[pg].index.str.endswith('unmatched')]) for pg in pg_order}
mtp_discrep_types_matched = {pg: count_discrep_types(mtp_discrep[pg].loc[~mtp_discrep[pg].index.str.endswith('unmatched')]) for pg in pg_order}

In [ ]:
fig = make_subplots(rows=2, cols=1,
                    subplot_titles=("De novo", "Map-to-pan"),
                   vertical_spacing=0.1)

fig.add_trace(go.Bar(name='PG-/HQ+', x=pg_order, y=[dn_discrep_types_matched[pg][0] for pg in pg_order], legendgroup='PG-/HQ+', marker_color='mediumpurple'), row=1, col=1)
fig.add_trace(go.Bar(name='PG+/HQ-', x=pg_order, y=[dn_discrep_types_matched[pg][1] for pg in pg_order], legendgroup='PG+/HQ-', marker_color='palegreen'), row=1, col=1)
fig.add_trace(go.Bar(name='PG-/HQ+', x=pg_order, y=[mtp_discrep_types_matched[pg][0] for pg in pg_order], showlegend=False, legendgroup='PG-/HQ+', marker_color='mediumpurple'), row=2, col=1)
fig.add_trace(go.Bar(name='PG+/HQ-', x=pg_order, y=[mtp_discrep_types_matched[pg][1] for pg in pg_order], showlegend=False, legendgroup='PG+/HQ-', marker_color='palegreen'), row=2, col=1)

# Change the bar mode
fig.update_layout(barmode='stack')
fig.update_yaxes(title_text = "# of discrepancies")
fig.update_layout(title_text="Number of discrepancies - matched genes")

fig.show()

In [ ]:
dn_discrep_matched_df = pd.DataFrame(dn_discrep_types_matched).transpose()
dn_discrep_matched_df.columns = ['PG-/HQ+','PG+/HQ-']
mtp_discrep_matched_df = pd.DataFrame(mtp_discrep_types_matched).transpose()
mtp_discrep_matched_df.columns = ['PG-/HQ+','PG+/HQ-']

discrep_matched_df = dn_discrep_matched_df.join(mtp_discrep_matched_df, rsuffix='MTP_')
discrep_matched_df.columns = pd.MultiIndex.from_product([['De novo','Map-to-pan'],['PG-/HQ+','PG+/HQ-']])
discrep_matched_df

### Discrepancies per gene

In [ ]:
dn_discrep_matched = {pg : dn_discrep[pg].dropna() for pg in dn_discrep}
mtp_discrep_matched = {pg : mtp_discrep[pg].dropna() for pg in mtp_discrep}

In [ ]:
dn_genes_with_x_discrep = pd.DataFrame({pg: dn_discrep_matched[pg].index.value_counts().value_counts() for pg in pg_order}).transpose().reset_index().melt(id_vars='index', value_vars=range(1,8))
dn_genes_with_x_discrep.columns = ['PG', 'discrepancies','genes']
mtp_genes_with_x_discrep = pd.DataFrame({pg: mtp_discrep_matched[pg].index.value_counts().value_counts() for pg in pg_order}).transpose().reset_index().melt(id_vars='index', value_vars=range(1,8))
mtp_genes_with_x_discrep.columns = ['PG', 'discrepancies','genes']

In [ ]:
fig = px.line(dn_genes_with_x_discrep, x='discrepancies', y='genes', color='PG', title="Discrepancies per gene - De novo")
fig.show()

In [ ]:
fig = px.line(mtp_genes_with_x_discrep, x='discrepancies', y='genes', color='PG', title="Discrepancies per gene - Map-to-pan")
fig.show()